# Saving a model trained with a tabular dataset in fast.ai
- Example of saving and reloading a model trained with a tabular dataset in fast.ai.
- This notebook is an extension of 

The example shown here is adapted from the paper by Howard and Gugger https://arxiv.org/pdf/2002.04688.pdf

# Prepare the notebook and ingest the dataset
The first section of this notebook is identical to the chapter 2 notebook for examining tabular curated datasets: https://github.com/PacktPublishing/Deep-Learning-with-fastai-Cookbook/blob/main/ch2/examining_tabular_datasets.ipynb

In [1]:
# imports for notebook boilerplate
!pip install -Uqq fastbook
import fastbook
from fastbook import *
from fastai.tabular.all import *


In [2]:
# set up the notebook for fast.ai
fastbook.setup_book()

In [3]:
# ingest the curated tabular dataset ADULT_SAMPLE
path = untar_data(URLs.ADULT_SAMPLE)

In [4]:
# examine the directory structure
path.ls()

(#3) [Path('/storage/data/adult_sample/export.pkl'),Path('/storage/data/adult_sample/adult.csv'),Path('/storage/data/adult_sample/models')]

In [5]:
# ingest the dataset into a Pandas dataframe
df = pd.read_csv(path/'adult.csv')

In [6]:
# examine the first few records in the dataframe
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [7]:
# get the number of records in the dataset
df.shape

(32561, 15)

In [8]:
# get the count of unique values in each column of the dataset
df.nunique()

age                  73
workclass             9
fnlwgt            21648
education            16
education-num        16
marital-status        7
occupation           15
relationship          6
race                  5
sex                   2
capital-gain        119
capital-loss         92
hours-per-week       94
native-country       42
salary                2
dtype: int64

In [9]:
# count the number of missing values in each column of the dataset
df.isnull().sum()

age                 0
workclass           0
fnlwgt              0
education           0
education-num     487
marital-status      0
occupation        512
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country      0
salary              0
dtype: int64

In [10]:
# get the subset of the dataset where age <= 40
# streetcarjan2014[streetcarjan2014.Location == "King and Shaw"].Route
df_young = df[df.age <= 40]
df_young.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
5,20,Private,63210,HS-grad,9.0,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,15,United-States,<50k
7,37,Private,138940,11th,7.0,Married-civ-spouse,NaN,Husband,White,Male,0,0,40,United-States,<50k
9,36,Self-emp-inc,216711,HS-grad,NaN,Married-civ-spouse,NaN,Husband,White,Male,99999,0,50,?,>=50k


# Define transforms, dependent variable, continuous and categorical columns
In this section we defined the transforms that will be applied to the dataset along with the target, continuous and categorical columns

In [11]:
# define transforms to apply to the tabular dataset
procs = [FillMissing,Categorify]
# define the dependent variable (y value)
dep_var = 'salary'
# define columns that are continuous / categorical
cont,cat = cont_cat_split(df, 1, dep_var=dep_var) 

# Define TabularDataLoaders object

In [12]:
# define TabularDataLoaders object
# valid_idx: the indices to use for the validation set
dls=TabularDataLoaders.from_df(df,path,procs= procs, 
                               cat_names= cat, cont_names = cont, y_names = dep_var, valid_idx=list(range(1024,1260)), bs=64)
                               

In [13]:
# use show_batch() to see a sample batch including x and y values
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,education-num_na,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary
0,Private,HS-grad,Never-married,Handlers-cleaners,Own-child,White,Male,Portugal,False,22.0,162667.0,9.0,0.0,0.0,50.0,<50k
1,Local-gov,HS-grad,Divorced,Protective-serv,Unmarried,White,Female,United-States,False,43.0,186995.0,9.0,0.0,0.0,40.0,<50k
2,Self-emp-not-inc,Some-college,Never-married,Craft-repair,Not-in-family,White,Female,United-States,False,18.0,42857.0,10.0,0.0,0.0,35.0,<50k
3,Private,HS-grad,Never-married,Other-service,Not-in-family,White,Male,United-States,False,40.0,209040.0,9.0,0.0,0.0,40.0,<50k
4,Private,Bachelors,Divorced,Adm-clerical,Not-in-family,White,Female,United-States,False,50.0,77905.0,13.0,0.0,0.0,8.0,<50k
5,?,Prof-school,Married-civ-spouse,?,Husband,White,Male,United-States,False,63.0,247986.0,15.0,0.0,0.0,30.0,>=50k
6,Private,12th,Never-married,Sales,Own-child,Black,Female,United-States,False,18.0,311795.0,8.0,0.0,0.0,20.0,<50k
7,Local-gov,Masters,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,False,43.0,174395.0,14.0,0.0,0.0,50.0,<50k
8,Self-emp-inc,HS-grad,Never-married,Sales,Not-in-family,White,Male,United-States,False,58.0,190541.0,9.0,0.0,0.0,47.0,<50k
9,Private,HS-grad,Married-civ-spouse,Other-service,Husband,White,Male,?,False,34.0,609789.0,9.0,0.0,0.0,30.0,<50k


# Define and train model

In [14]:
learn = tabular_learner(dls,layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.328067,0.376390,0.830508,00:07
1,0.324054,0.356516,0.830508,00:07
2,0.300545,0.340962,0.838983,00:07


In [15]:
# show sample result, including transformed x, y and predicted transformed y
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,education-num_na,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary,salary_pred
0,7.0,12.0,3.0,13.0,1.0,5.0,2.0,40.0,1.0,58.0,248841.0,9.0,15024.0,0.0,40.0,1.0,1.0
1,8.0,11.0,3.0,11.0,1.0,5.0,2.0,40.0,1.0,41.0,309056.0,16.0,0.0,0.0,50.0,1.0,1.0
2,5.0,5.0,7.0,9.0,3.0,2.0,1.0,31.0,1.0,63.0,106910.0,3.0,0.0,0.0,19.0,0.0,0.0
3,1.0,11.0,3.0,1.0,1.0,5.0,2.0,40.0,1.0,72.0,118902.0,16.0,0.0,2392.0,6.0,1.0,1.0
4,5.0,2.0,5.0,7.0,3.0,3.0,1.0,40.0,1.0,41.0,155657.0,7.0,0.0,0.0,40.0,0.0,0.0
5,5.0,2.0,1.0,8.0,2.0,5.0,2.0,40.0,1.0,38.0,252250.0,7.0,0.0,0.0,65.0,0.0,0.0
6,5.0,16.0,5.0,15.0,5.0,5.0,2.0,40.0,1.0,29.0,277342.0,10.0,0.0,0.0,40.0,0.0,0.0
7,5.0,12.0,3.0,7.0,1.0,5.0,2.0,40.0,1.0,25.0,218667.0,9.0,0.0,0.0,40.0,0.0,0.0
8,7.0,10.0,1.0,5.0,2.0,5.0,2.0,40.0,1.0,54.0,154785.0,13.0,0.0,0.0,50.0,0.0,0.0


# Examine the structure of the trained model structure

Use the summary() function to see the structure of the trained model, including:

- the layers that make up the model
- total parameters
- loss function
- optimizer function
- callbacks



In [16]:
learn.summary()

TabularModel (Input shape: ['64 x 9', '64 x 6'])
Layer (type)         Output Shape         Param #    Trainable 
Embedding            64 x 6               60         True      
________________________________________________________________
Embedding            64 x 8               136        True      
________________________________________________________________
Embedding            64 x 5               40         True      
________________________________________________________________
Embedding            64 x 8               128        True      
________________________________________________________________
Embedding            64 x 5               35         True      
________________________________________________________________
Embedding            64 x 4               24         True      
________________________________________________________________
Embedding            64 x 3               9          True      
_________________________________________________

# Save the trained model

In [17]:
learn.path

Path('/storage/data/adult_sample')

In [18]:
print(os.getcwd())

/notebooks/fastai2/Deep-Learning-with-fastai-Cookbook/ch3


In [19]:
if 'google.colab' in str(get_ipython()):
    temp_path = Path('/content/gdrive/MyDrive/fastai_cookbook/Deep-Learning-with-fastai-Cookbook/ch3/')
else:
    temp_path = Path(os.getcwd())
learn.path = temp_path

In [20]:
# save the trained model in /storage/data/adult_sample/adult_sample_model.pkl
learn.export('adult_sample_model.pkl')

In [21]:
learn2 = load_learner(os.path.join(temp_path,'adult_sample_model.pkl'))

In [22]:
sample_URL = 'https://raw.githubusercontent.com/PacktPublishing/Deep-Learning-with-fastai-Cookbook/main/ch3/adult_sample_test.csv'
df_test = pd.read_csv(sample_URL)

In [23]:
df_test.shape

(128, 14)

In [24]:
df_test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States


In [25]:
df_test.iloc[1]

age                             44
workclass                  Private
fnlwgt                      236746
education                  Masters
education-num                   14
marital-status            Divorced
occupation         Exec-managerial
relationship         Not-in-family
race                         White
sex                           Male
capital-gain                 10520
capital-loss                     0
hours-per-week                  45
native-country       United-States
Name: 1, dtype: object

In [26]:
df_test.iloc[0].shape

(14,)

In [27]:
df_test.dtypes

age                 int64
workclass          object
fnlwgt              int64
education          object
education-num     float64
marital-status     object
occupation         object
relationship       object
race               object
sex                object
capital-gain        int64
capital-loss        int64
hours-per-week      int64
native-country     object
dtype: object

In [28]:
test_sample = df_test.iloc[1]
learn2.predict(test_sample)

(   workclass  education  marital-status  occupation  relationship  race  sex  \
 0        5.0       13.0             1.0         5.0           2.0   5.0  2.0   
 
    native-country  education-num_na   age    fnlwgt  education-num  \
 0            40.0               1.0  44.0  236746.0           14.0   
 
    capital-gain  capital-loss  hours-per-week  salary  
 0       10520.0           0.0            45.0     1.0  ,
 tensor(1),
 tensor([0.2023, 0.7977]))

In [29]:
pred_class,pred_idx,outputs = learn2.predict(test_sample)
print("pred class is: ",pred_class)

pred class is:     workclass  education  marital-status  occupation  relationship  race  sex  \
0        5.0       13.0             1.0         5.0           2.0   5.0  2.0   

   native-country  education-num_na   age    fnlwgt  education-num  \
0            40.0               1.0  44.0  236746.0           14.0   

   capital-gain  capital-loss  hours-per-week  salary  
0       10520.0           0.0            45.0     1.0  


In [30]:
type(pred_class)

fastai.tabular.core.TabularPandas

In [31]:
pred_class['workclass']

0    5.0
Name: workclass, dtype: float64